Nesse experimento um ECoG registra o sinal de EEG de um macaco em três situações: 

- Olhos abertos
- Olhos fechados
- Anestesiado

O ECoG usa 128 canais, cada canal é um arquivo matlab, a taxa de amostragem é de 1000Hz e o experimento foi dividido em 2 partes. Na primeira o macaco está de olhos abertos e fechados e na segunda parte é aplicada anestesia. O arquivo ECoGTime.mat possui a variável tempo apenas. O Condition.mat possui 3 variáveis: ConditionLabel, ConditionIndex e ConditionTime. O ConditionLabel tem os nomes da condição que o macaco se encontra e ConditionIndex e ConditionTime indicam o início e o final de cada condição.


In [2]:
import scipy.io as sio
import matplotlib.pyplot as plt
import warnings

import numpy as np
from scipy.io import loadmat 
from nitime.algorithms import multi_taper_psd
from nitime.utils import dpss_windows
import statsmodels.api as sm
from statsmodels.genmod.families import Poisson
import statsmodels.formula.api as smf
from pandas import DataFrame as df
from pandas import concat
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import chi2, norm
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, xlim, ylim, show, imshow, xlabel, ylabel, bar, figure

from matplotlib.pyplot import xlabel, ylabel, plot, show, title
from matplotlib import rcParams
from numpy import hanning, where, histogram, arange
rcParams['figure.figsize'] = (12,3)


In [ ]:
#Questão 1




# Qual a duração total de cada condição ? 







#Qual a resolução da frequência  e a frequência de Nyquist?

In [ ]:
#Questão 2

#Crie um plot para cada condição e descreva se você consegue enxergar algum ritmo. Plote apenas um intervalo curto se for necessário.

In [ ]:
#Questão 3
#Plote a autocovariância para cada condição.

In [ ]:
#Questão 4
#Plote o espectro para cada condição. 
#Existe a necessidade de passar algum filtro? Provavelmente terá 60 Hz nos canais, caso tenha, passe um filtro e calcule novamente o espectro.

In [ ]:
#Questão 5
#Existe diferenças espectrais entre as condições?

In [ ]:
#Questão 6
#Calcule o espectrograma para todo o sinal, os tempos das mudanças de condições batem com o que foi relatado no ConditionTime?

In [ ]:
#Questão 7
#Calcule os espectros usando o método multitaper. Justifique sua escolha do número de tapers em relação à resolução da frequência. Alguma nova frequência ficou aparente?

In [ ]:
#Questão 8
#Para cada condição, divida o sinal em trechos de 1s, calcule a coerência entre os eletrodos. Existe mudança entre acoplamento entre os eletrodos?